#Airbnb Geneva Analysis

## Problem Statement:
With the rise of the sharing economy, Airbnb has become one of the leading platforms in the travel industry, especially in metropolitan cities like Geneva. However, there's much speculation and curiosity regarding what makes an Airbnb listing successful, particularly in terms of overall ratings. Our primary question is:

"How do different neighborhoods compare in terms of average price, overall rating, and property types?"

##  Objectives and Deliverables:
The research will deliver insights on the following:

- Factors for higher rating: Investigate property types in Geneva and understand their influence on ratings and prices. The aim is to decipher which property types tend to receive better ratings and why.

- Neighborhood Comparison: break down and compare different neighborhoods in Geneva regarding their average price, overall rating, and most common property types. This will provide insights into which neighborhoods are considered premium and why.

- Success Factors for Listings: Understand the correlation between price and overall rating. The age-old question – does higher price mean better ratings?

## Data Source:
The dataset is sourced from Kaggle and represents Airbnb listings for Geneva as of 29th June 2021.

Link to the dataset: https://www.kaggle.com/datasets/prayankkul/airbnb-geneva-switzerland-29-jun-2021/data

## ETL Pipeline Overview:
The ETL process consists of the following steps:

Extract: Download the data from the provided Kaggle link and read it into Databricks.

Transform:

- Cleanse the data by removing nulls, duplicates, and outliers where necessary.
- Transform data types to the suitable format.
- Engineer new features such as 'month' for new listings and reviews to facilitate growth metrics.
- Aggregate data at various levels like property type and neighborhoods for analysis.
- Join datasets as necessary, for instance, merging listings data with reviews data.
- Load: Data will be loaded into Delta tables and views for optimized querying and analysis in Databricks.

## Data Cube Design:
###Dimensions:

- Listing details (property type, amenities, host information).
- Review details (date, reviewer details).
- Neighbourhood information.

###Measures:

- Average Rating.
- Average Price.
- Count of Listings.
- Count of Reviews.

## Visualization and Reporting:
Using Databricks' built-in visualizations, we will create various plots and graphs to visually depict:

The distribution of ratings across different property types.
Average prices and ratings across neighborhoods.
Growth charts depicting the monthly trend of new listings and reviews.
Scatter plots to explore the correlation between price and rating.
7. Conclusion:
Our analysis aims to provide a comprehensive understanding of what contributes to a successful Airbnb listing in Geneva, giving stakeholders actionable insights into property optimization, pricing strategies, and more.

## Limitations:
- Compute Constraints: Using the Databricks community edition imposes a significant limitation in terms of available compute. This results in slower data processing times, affecting the efficiency of the overall ETL process.

- Transient Compute Instances: After every run, a new compute instance must be generated, causing disruptions in continuity and adding overhead in terms of setup time.

- Delta File Management: As the cluster changes after each run, delta files associated with tables have to be manually removed. This introduces potential data loss risks and requires additional manual oversight.

- SQL Errors:  Table constraints are not supported, limiting the schema enforcement capabilities, which is essential for data quality and integrity.

- Cluster Management: The inability to restart the same cluster, even though the files are retained, creates inconsistencies in processing and potential mismatches in data.

## Proposed Imporovement
- Enhanced Alerting and Monitoring: Databricks offers powerful alerting and monitoring features in its full version. By leveraging these capabilities, we can:

- Automated Alerts: Configure Databricks to send alerts in case of anomalies in data, failures in the ETL process, or any issues with the compute resources. This immediate notification ensures timely interventions and minimal disruptions.

- Log Analysis: Capture and analyze logs for the ETL processes. Any discrepancies, errors, or inefficiencies can be traced back to their root cause using detailed logs.

- Error Handling Mechanisms:
Incorporate error-handling mechanisms in the ETL scripts. This ensures graceful failure in the event of SQL errors, providing more informative messages and possible resolutions.

- Historization: 
Implementing Slowly Changing Dimensions (SCD) to track historical changes, providing a more comprehensive view of how data evolves over time.


- Normalization: 
the star schema is excellent for querying, normalizing to at least 2NF to ensure data consistency and reduce redundancy.

- Data Quality Checks: 
Incorporate regular data quality checks, ensuring that data in the warehouse remains accurate, consistent, and up-to-date.

In [0]:
#clean up resources when starting with a new cluster
#dbutils.fs.ls("dbfs:/user/hive/warehouse/") 

Creation of multiple databases: RAW, STAGING, DWH and  REPORTING 
Organizing data into different databases to streamline the data engineering pipeline. Raw data is kept untouched for auditing and traceability; staging for transformations; DWH for structured and optimized data storage; Reporting for user-specific views;

In [0]:
%sql
--CREATE DATABASE
CREATE DATABASE RAW;
CREATE DATABASE STAGING;
CREATE DATABASE DWH;
CREATE DATABASE REPORTING;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.analysis.NamespaceAlreadyExistsException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `RAW` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.
	at org.apache.spark.sql.errors.QueryCompilationErrors$.namespaceAlreadyExistsError(QueryCompilationErrors.scala:1544)
	at org.apache.spark.sql.execution.datasources.v2.CreateNamespaceExec.run(CreateNamespaceExec.scala:53)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.$anonfun$result$1(V2CommandExec.scala:47)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:47)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:45)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:54)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.$anonfun$applyOrElse$3(QueryExecution.scala:287)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:166)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.$anonfun$applyOrElse$2(QueryExecution.scala:287)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId0$8(SQLExecution.scala:301)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:555)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId0$1(SQLExecution.scala:204)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1117)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId0(SQLExecution.scala:153)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:497)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.$anonfun$applyOrElse$1(QueryExecution.scala:286)
	at org.apache.spark.sql.execution.QueryExecution.org$apache$spark$sql$execution$QueryExecution$$withMVTagsIfNecessary(QueryExecution.scala:260)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.applyOrElse(QueryExecution.scala:281)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.applyOrElse(QueryExecution.scala:266)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:470)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:470)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDownWithPruning(LogicalPlan.scala:36)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning(AnalysisHelper.scala:316)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning$(AnalysisHelper.scala:312)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDownWithPruning(LogicalPlan.scala:36)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDownWithPruning(LogicalPlan.scala:36)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:446)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$eagerlyExecuteCommands$1(QueryExecution.scala:266)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:372)
	at org.apache.spark.sql.execution.QueryExecution.eagerlyExecuteCommands(QueryExecution.scala:266)
	at org.apache.spark.sql.execution.QueryExecution.commandExecuted$lzycompute(QueryExecu

###RAW TABLES
- RAW.reviews
- RAW.listing
- RAW.calenar

Copies data from the '/FileStore/tables/raw/calendar/**' directory into table, using specified formatting options.

####RAW.calendar

In [0]:
%sql
--in built won't copy duplicates, would only process newer files  
CREATE TABLE IF NOT EXISTS raw.calendar;
 
COPY INTO raw.calendar
  FROM '/FileStore/tables/raw/calendar/**'
  FILEFORMAT = CSV
  FORMAT_OPTIONS ('inferSchema' = 'true',
                  'mergeSchema' = 'true',
                  'delimiter' = ',',
                  'header' = 'true',
                  'quote' ='"')
  COPY_OPTIONS ('mergeSchema' = 'true',
                'ignoreCorruptFiles' = 'true');

####RAW.reviews

PySpark reads from the CSV file from /FileStore/tables/raw/reviews/** containing reviews into a DataFrame df.
Applies a schema to the data.
Adjusts whitespace from the columns and writes this DataFrame to the table RAW.reviews.
processing with pyspark as the file contains some corrupted data

In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import functions as F

# Adjust the schema to match the new dataset
schema = StructType([
    StructField("listing_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("reviewer_id", StringType(), True),
    StructField("reviewer_name", StringType(), True),
    StructField("comments", StringType(), True)
])

# File location and type
file_location = "/FileStore/tables/raw/reviews/**"  # Adjust the file path
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .schema(schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("multiline", "true") \
  .option("quote", "\"") \
  .option("escape", "\"") \
  .load(file_location)

def trim_whitespace(df):
    for col in df.columns:
        df = df.withColumn(col, F.trim(df[col]))
    return df

# Apply the function to DataFrame
df = trim_whitespace(df)

# Write dataframe to raw table
df.write.mode("overwrite").saveAsTable("RAW.reviews")


#### RAW.LISTING

Similar to above immediate cleaning and ingestion help maintain a consistent pipeline. CSV file in "/FileStore/tables/raw/listing/**" is written to RAW.listing table 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

def trim_whitespace(df):
    for col in df.columns:
        df = df.withColumn(col, F.trim(df[col]))
    return df

# File attributes
file_location = "/FileStore/tables/raw/listing/**"
file_type = "csv"

# CSV read options
options = {
    "inferSchema": "true",
    "header": "true",
    "sep": ",",
    "multiline": "true",
    "quote": "\"",
    "escape": "\""
}

# Read the data
df = spark.read.format(file_type).options(**options).load(file_location)

# Trim whitespace from all columns
df = trim_whitespace(df)

# Write dataframe to raw table
df.write.mode("overwrite").saveAsTable("RAW.listing")


###STAGING TABLES
Sets up staging tables for calendar, reviews, and listing with various transformations.
These staging tables represent a "cleaned" version of raw data. Transformations at this stage mean that downstream tasks (like analytics) won't need to deal with raw quirks or errors, optimizing the pipeline and reducing downstream work.
- STAGING.CALENDAR
- STAGING.REVIEWS
- STAGING.LISTING
- STAGING.HOST

I went with the truncate and Insert Logic in Staging: Ensuring a fresh start for each data load, maintain data integrity, and provide a potential performance boost. 

###validation:
Duplicate Checks: Ensuring data quality by frequently checking for duplicates. Duplicates can cause inflated metrics and misinterpretations in later analysis.


#### CALENDAR

In [0]:
%sql
--create staging table
CREATE OR REPLACE TABLE STAGING.CALENDAR (
  LISTING_ID INTEGER,
  DATE DATE,
  AVAILABLE BOOLEAN,
  PRICE FLOAT,
  ADJUSTED_PRICE FLOAT,
  MINIMUM_NIGHTS INTEGER,
  MAXIMUM_NIGHTS INTEGER,
  LOAD_ID STRING,
  RECORD_HASH STRING
)


In [0]:
%sql
--STAGING TABLE
TRUNCATE TABLE Staging.calendar;

INSERT INTO staging.calendar
SELECT
 DISTINCT 
    listing_id,
    CAST(date AS DATE) AS date,
    CASE WHEN available = 't' THEN TRUE 
         WHEN available = 'f' THEN FALSE 
         ELSE NULL END AS available,
    CAST(REPLACE(price, '$', '') AS DECIMAL(10, 2)) AS price,
    CAST(REPLACE(adjusted_price, '$', '') AS DECIMAL(10, 2)) AS adjusted_price,
    minimum_nights,
    maximum_nights,
    current_timestamp() AS LOAD_ID,
    MD5(CONCAT(CAST(RAND() AS STRING), CAST(CURRENT_TIMESTAMP AS STRING))) AS record_hash
FROM 
  raw.calendar;


In [0]:
%sql
--duplicates check
select listing_id, date, count(*) 
from staging.calendar
group by all
having count(*) >1

####Reviews

In [0]:
%sql
--create staging table
CREATE OR REPLACE TABLE STAGING.REVIEWS (
  LISTING_ID INTEGER,
  REVIEW_ID INTEGER,
  REVIEWER_ID INTEGER,
  REVIEWER_NAME STRING,
  REVIEW_DATE DATE,
  COMMENTS STRING,
  LOAD_ID STRING,
  RECORD_HASH STRING
);

In [0]:
%sql
--STAGING TABLE
TRUNCATE TABLE Staging.REVIEWS;

INSERT INTO staging.REVIEWS
SELECT
 DISTINCT
 CAST(LISTING_ID AS INTEGER) AS LISTING_ID,
 cast(review_id as INTEGER) AS REVIEW_ID,
 cast(REVIEWER_ID as INTEGER) AS REVIEWER_ID,
 REVIEWER_NAME STRING,
 CAST(date AS DATE) AS review_date,
 COMMENTS,
 REPLACE(CAST(current_timestamp() AS STRING), '-', '') AS LOAD_ID,
 MD5(CONCAT(CAST(RAND() AS STRING), CAST(CURRENT_TIMESTAMP AS STRING))) as record_hash
FROM 
  raw.REVIEWS;


In [0]:
%sql
--duplicates check
select review_id, listing_id, reviewer_id, count(*)
from staging.review
group by all
having count(*) > 1

#### LISTING

In [0]:
%sql
--create staging table
CREATE TABLE Staging.LISTING (
    listing_id INTEGER,
    listing_url STRING,
    name STRING,
    description STRING,
    neighborhood_overview STRING,
    picture_url STRING,
    host_id INTEGER,
    host_url STRING,
    host_name STRING,
    host_since_date DATE,
    host_location STRING,
    host_about STRING,
    host_response_time STRING,
    host_response_rate_percent FLOAT,
    host_acceptance_rate_percent FLOAT,
    host_is_superhost BOOLEAN,
    host_thumbnail_url STRING,
    host_picture_url STRING,
    host_neighbourhood STRING,
    host_listings_count INTEGER,
    host_total_listings_count INTEGER,
    host_verifications STRING,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood STRING,
    neighbourhood_cleansed STRING,
    latitude FLOAT,
    longitude FLOAT,
    property_type STRING,
    room_type STRING,
    accommodates INTEGER,
    bathrooms_text STRING,
    bedrooms INTEGER,
    beds INTEGER,
    amenities STRING,
    price DECIMAL(10, 2),
    minimum_nights INTEGER,
    maximum_nights INTEGER,
    minimum_minimum_nights INTEGER,
    maximum_minimum_nights INTEGER,
    minimum_maximum_nights INTEGER,
    maximum_maximum_nights INTEGER,
    minimum_nights_avg_ntm FLOAT,
    maximum_nights_avg_ntm FLOAT,
    has_availability BOOLEAN,
    availability_30 INTEGER,
    availability_60 INTEGER,
    availability_90 INTEGER,
    availability_365 INTEGER,
    number_of_reviews INTEGER,
    number_of_reviews_ltm INTEGER,
    number_of_reviews_l30d INTEGER,
    first_review_date DATE,
    last_review_date DATE,
    review_scores_rating FLOAT,
    review_scores_accuracy FLOAT,
    review_scores_cleanliness FLOAT,
    review_scores_checkin FLOAT,
    review_scores_communication FLOAT,
    review_scores_location FLOAT,
    review_scores_value FLOAT,
    license STRING,
    instant_bookable BOOLEAN,
    calculated_host_listings_count INTEGER,
    calculated_host_listings_count_entire_homes INTEGER,
    calculated_host_listings_count_private_rooms INTEGER,
    calculated_host_listings_count_shared_rooms INTEGER,
    reviews_per_month FLOAT,
    LOAD_ID STRING,
    record_hash STRING
);



In [0]:
%sql
--STAGING TABLE
TRUNCATE TABLE Staging.LISTING;

INSERT INTO staging.listing
SELECT
    cast(id as INTEGER) as listing_id,
    listing_url,
    name,
    description,
    neighborhood_overview,
    picture_url,
    cast(host_id as INTEGER) as host_id,
    host_url,
    host_name,
    CAST(host_since AS DATE) AS host_since_date,
    host_location,
    host_about,
    host_response_time,
    CASE WHEN host_response_rate LIKE '%%%' THEN CAST(REPLACE(host_response_rate, '%', '') AS FLOAT) / 100 ELSE NULL END AS host_response_rate_percent,
    CASE WHEN host_acceptance_rate LIKE '%%%' THEN CAST(REPLACE(host_acceptance_rate, '%', '') AS FLOAT) / 100 ELSE NULL END AS host_acceptance_rate_percent,
    CASE WHEN host_is_superhost = 't' THEN TRUE 
         WHEN host_is_superhost = 'f' THEN FALSE 
         ELSE NULL END AS host_is_superhost,
    host_thumbnail_url,
    host_picture_url,
    host_neighbourhood,
    CAST(host_listings_count AS INT) AS host_listings_count,
    CAST(host_total_listings_count AS INT) AS host_total_listings_count,
    host_verifications,
    CASE WHEN host_has_profile_pic = 't' THEN TRUE 
         WHEN host_has_profile_pic = 'f' THEN FALSE
         ELSE NULL END AS host_has_profile_pic,
    CASE WHEN host_identity_verified = 't' THEN TRUE 
         WHEN host_identity_verified = 'f' THEN FALSE 
         ELSE NULL END AS host_identity_verified,
    neighbourhood,
    neighbourhood_cleansed,
    CAST(latitude AS FLOAT) AS latitude,
    CAST(longitude AS FLOAT) AS longitude,
    property_type,
    room_type,
    CAST(accommodates AS INT) AS accommodates,
    bathrooms_text,
    CAST(bedrooms AS INT) AS bedrooms,
    CAST(beds AS INT) AS beds,
    amenities,
    CAST(REPLACE(price, '$', '') AS DECIMAL(10, 2)) AS price,
    CAST(minimum_nights AS INT) AS minimum_nights,
    CAST(maximum_nights AS INT) AS maximum_nights,
    CAST(minimum_minimum_nights AS INT) AS minimum_minimum_nights,
    CAST(maximum_minimum_nights AS INT) AS maximum_minimum_nights,
    CAST(minimum_maximum_nights AS INT) AS minimum_maximum_nights,
    CAST(maximum_maximum_nights AS INT) AS maximum_maximum_nights,
    CAST(minimum_nights_avg_ntm AS FLOAT) AS minimum_nights_avg_ntm,
    CAST(maximum_nights_avg_ntm AS FLOAT) AS maximum_nights_avg_ntm,
    CASE WHEN has_availability = 't' THEN TRUE
         WHEN has_availability = 'f' THEN FALSE 
         ELSE NULL END AS has_availability,
    CAST(availability_30 AS INT) AS availability_30,
    CAST(availability_60 AS INT) AS availability_60,
    CAST(availability_90 AS INT) AS availability_90,
    CAST(availability_365 AS INT) AS availability_365,
    CAST(number_of_reviews AS INT) AS number_of_reviews,
    CAST(number_of_reviews_ltm AS INT) AS number_of_reviews_ltm,
    CAST(number_of_reviews_l30d AS INT) AS number_of_reviews_l30d,
    CAST(first_review AS DATE) AS first_review_date,
    CAST(last_review AS DATE) AS last_review_date,
    CAST(review_scores_rating AS FLOAT) AS review_scores_rating,
    CAST(review_scores_accuracy AS FLOAT) AS review_scores_accuracy,
    CAST(review_scores_cleanliness AS FLOAT) AS review_scores_cleanliness,
    CAST(review_scores_checkin AS FLOAT) AS review_scores_checkin,
    CAST(review_scores_communication AS FLOAT) AS review_scores_communication,
    CAST(review_scores_location AS FLOAT) AS review_scores_location,
    CAST(review_scores_value AS FLOAT) AS review_scores_value,
    license,
    CASE WHEN instant_bookable = 't' THEN TRUE 
         WHEN instant_bookable = 'f' THEN FALSE
         ELSE NULL END AS instant_bookable,
    CAST(calculated_host_listings_count AS INT) AS calculated_host_listings_count,
    CAST(calculated_host_listings_count_entire_homes AS INT) AS calculated_host_listings_count_entire_homes,
    CAST(calculated_host_listings_count_private_rooms AS INT) AS calculated_host_listings_count_private_rooms,
    CAST(calculated_host_listings_count_shared_rooms AS INT) AS calculated_host_listings_count_shared_rooms,
    CAST(reviews_per_month AS FLOAT) AS reviews_per_month,
    REPLACE(CAST(current_timestamp() AS STRING), '-', '') AS LOAD_ID,
    MD5(CONCAT(CAST(RAND() AS STRING), CAST(CURRENT_TIMESTAMP AS STRING))) as record_hash
FROM 
    raw.listing;


In [0]:
%sql
--check for duplicate records
select listing_id,count(*) 
from staging.listing
group by all
having count(*) > 1;

####HOST

In [0]:
%sql
--create staging table
CREATE OR REPLACE TABLE staging.host(
    host_id INT,
    host_name STRING,
    host_since_date DATE,
    host_location STRING,
    host_about STRING,
    host_response_time STRING,
    host_response_rate_percent DOUBLE,
    host_acceptance_rate_percent DOUBLE,
    host_is_superhost BOOLEAN,
    host_thumbnail_url STRING,
    host_picture_url STRING,
    host_neighbourhood STRING,
    host_verifications STRING,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    LOAD_ID STRING,
    record_hash STRING
);


 Inserts unique host data from staging.listing into staging.host.

utilizing window functions and ranking, to ensure only the most recent and unique data about hosts is ingested. 

In [0]:
%sql
TRUNCATE TABLE staging.host;

-- Insert data into staging.host from staging.listing
INSERT INTO staging.host
WITH RankedData AS (
  SELECT 
    host_id,
    host_name,
    host_since_date,
    host_location,
    host_about,
    host_response_time,
    host_response_rate_percent,
    host_acceptance_rate_percent,
    host_is_superhost,
    host_thumbnail_url,
    host_picture_url,
    host_neighbourhood,
    host_verifications,
    host_has_profile_pic,
    host_identity_verified,
    CAST(current_timestamp() AS STRING) AS LOAD_ID,
    MD5(CONCAT(CAST(RAND() AS STRING), CAST(CURRENT_TIMESTAMP AS STRING))) as record_hash,
    ROW_NUMBER() OVER(PARTITION BY host_id ORDER BY LOAD_ID DESC) as rn
  FROM staging.listing
  WHERE host_id IS NOT NULL
)
SELECT 
    host_id,
    host_name,
    host_since_date,
    host_location,
    host_about,
    host_response_time,
    ifnull(host_response_rate_percent,0 ),
    ifnull(host_acceptance_rate_percent, 0),
    host_is_superhost,
    host_thumbnail_url,
    host_picture_url,
    host_neighbourhood,
    host_verifications,
    host_has_profile_pic,
    host_identity_verified,
    LOAD_ID,
    record_hash
FROM RankedData
WHERE rn = 1;


In [0]:
%sql
--check for duplicate records
select host_id,count(*) 
from staging.host
group by all
having count(*) > 1;


###DIMENSION AND FACT TABLE (DWH)
The DWH layer uses the MERGE INTO statement in conjunction with EDW_PUBLCN_ID to ensure idempotency during data ingestion and updates. EDW_PUBLCN_ID, typically a timestamp or unique identifier for each batch of data processed, ensures that each batch can be re-ingested or re-run without introducing unwanted duplicates. If the same data point is re-introduced, the MERGE INTO statement checks against unique identifiers (like listing_id) and uses the EDW_PUBLCN_ID to determine whether to update or retain the existing data. This approach ensures that the DWH remains consistent and accurate, even in the face of repeated data processing or potential reprocessing scenarios.
tables in this db includes:
- dim_date
- dim_listing
- dim_host
- dim_listing_calendar
- dim_neighbourhood
- fct_ratings_review
- fct_reviews

####Creation of Dim_Date
which holds various date-related attributes for our data warehouse. This table is important for time-based analyses.
Attributes:
- date_id: Unique identifier for each date.
- date: Actual date value.
- year: Year extracted from the date.
- quarter: Quarterly segmentation of the year.
- month: Numeric month value.
- month_name: Name of the month.
- day: Day of the month.
- weekday: Day of the week.

In [0]:
%sql
-- Create the Dim_Date table
CREATE OR REPLACE TABLE dwh.Dim_Date (
    date_id INT NOT NULL, -- PRIMARY KEY, Table constraints are only supported in Unity Catalog.
    date DATE,
    year INT,
    quarter INT,
    month INT,
    month_name STRING,
    day INT,
    weekday STRING
);

Dim_Date is populated using a CTE that extracts unique dates from the`staging.calendar` table. 

In [0]:
%sql
---- Updating Dim_Date
WITH Date_CTE AS (
    SELECT DISTINCT date
    FROM staging.calendar
)

INSERT INTO dwh.Dim_Date
SELECT 
  ROW_NUMBER() OVER (ORDER BY date) as date_id,
  date,
  EXTRACT(YEAR FROM date) as year,
  CAST(EXTRACT(MONTH FROM date) / 4 + 1 AS INT) as quarter, 
  EXTRACT(MONTH FROM date) as month,
  DATE_FORMAT(date, 'MMMM') as month_name,
  EXTRACT(DAY FROM date) as day,
  DATE_FORMAT(date, 'EEEE') as weekday
FROM 
  Date_CTE;


#### DIM_HOST
the attributes (columns) of the `Dim_Host` table are as follows:

- `host_id` - INT: Identifier for the host.
- `host_name` - STRING: Name of the host.
- `host_since_date` - DATE: The date when the host joined.
- `host_location` - STRING: Location of the host.
- `host_about` - STRING: Description or information about the host.
- `host_response_time` - STRING: Estimated response time of the host to queries or bookings.
- `host_response_rate_percent` - DOUBLE: Percentage rate at which the host responds to queries or bookings.
- `host_acceptance_rate_percent` - DOUBLE: Percentage rate at which the host accepts booking requests.
- `host_is_superhost` - BOOLEAN: Flag to indicate if the host is a superhost.
- `host_thumbnail_url` - STRING: URL for the host's thumbnail picture.
- `host_picture_url` - STRING: URL for the host's main picture.
- `host_neighbourhood` - STRING: The neighborhood where the host's properties are located, or where the host resides.
-. `host_verifications` - STRING: Methods the host has used to verify their identity.
- `host_has_profile_pic` - BOOLEAN: Flag to indicate if the host has a profile picture.
- `host_identity_verified` - BOOLEAN: Flag to indicate if the host's identity has been verified.
- `EDW_PUBLCN_ID` - TIMESTAMP: Timestamp indicating when the record was published or last updated in the Enterprise Data Warehouse (EDW).

The table is partitioned by the `host_location` attribute.

In [0]:
%sql
--create the dwh.host table
CREATE OR REPLACE TABLE dwh.dim_host(
    host_id INT NOT NULL,
    host_name STRING,
    host_since_date DATE,
    host_location STRING,
    host_about STRING,
    host_response_time STRING,
    host_response_rate_percent DOUBLE,
    host_acceptance_rate_percent DOUBLE,
    host_is_superhost BOOLEAN,
    host_thumbnail_url STRING,
    host_picture_url STRING,
    host_neighbourhood STRING,
    host_verifications STRING,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    EDW_PUBLCN_ID TIMESTAMP
)
USING DELTA
PARTITIONED BY (host_location);


Updating Dim_Host
we're merging data from the staging.host table into our dwh.dim_host table. The goal is to keep the dimension table up-to-date

In [0]:
%sql
--DIM_HOST
MERGE INTO dwh.dim_host AS target
USING staging.host AS source
ON target.host_id = source.host_id
WHEN MATCHED THEN
   UPDATE SET 
       target.host_name = source.host_name,
       target.host_location = source.host_location,
       target.host_since_date = source.host_since_date,
       target.host_about = source.host_about,
       target.host_response_time = source.host_response_time,
       target.host_response_rate_percent = source.host_response_rate_percent,
       target.host_acceptance_rate_percent = source.host_acceptance_rate_percent,
       target.host_is_superhost = source.host_is_superhost,
       target.host_thumbnail_url = source.host_thumbnail_url,
       target.host_picture_url = source.host_picture_url,
       target.host_neighbourhood = source.host_neighbourhood,
       target.host_verifications = source.host_verifications,
       target.host_has_profile_pic = source.host_has_profile_pic,
       target.host_identity_verified = source.host_identity_verified,
       target.EDW_PUBLCN_ID = current_timestamp()
WHEN NOT MATCHED THEN
   INSERT (
       host_id, 
       host_name, 
       host_location, 
       host_since_date, 
       host_about,
       host_response_time,
       host_response_rate_percent,
       host_acceptance_rate_percent,
       host_is_superhost,
       host_thumbnail_url,
       host_picture_url,
       host_neighbourhood,
       host_verifications,
       host_has_profile_pic,
       host_identity_verified,
       EDW_PUBLCN_ID
   )
   VALUES (
       source.host_id, 
       source.host_name, 
       source.host_location, 
       source.host_since_date, 
       source.host_about,
       source.host_response_time,
       source.host_response_rate_percent,
       source.host_acceptance_rate_percent,
       source.host_is_superhost,
       source.host_thumbnail_url,
       source.host_picture_url,
       source.host_neighbourhood,
       source.host_verifications,
       source.host_has_profile_pic,
       source.host_identity_verified,
       current_timestamp()
   );


In [0]:
%sql
--check for duplicate records
select host_id,count(*) 
from dwh.dim_host
group by all
having count(*) > 1;

####dim_neighbourhood 

The dwh.dim_neighbourhood table contains:

- neighbourhood_id: A unique sequential ID for neighbourhoods.
- neighbourhood_name: The neighbourhood name.
- neighborhood_overview: A representative overview of the neighbourhood.
- latitude:  the latitude of the listing's location.
- longitude: the longitude of the listing's location.
- EDW_PUBLCN_ID: A timestamp indicating when the record was inserted.

In [0]:
%sql
CREATE OR REPLACE TABLE dwh.dim_neighbourhood (
    neighbourhood_id INT,
    neighbourhood_name STRING,
    neighborhood_overview STRING,
    latitude DOUBLE,
    longitude DOUBLE,
    EDW_PUBLCN_ID TIMESTAMP
) USING DELTA;


In [0]:
%sql
--identify potential new neighbourhoods
WITH NewNeighbourhoods AS (
    SELECT
        neighbourhood_cleansed as neighbourhood_name,
        MAX(neighborhood_overview) AS neighborhood_overview,
        AVG(latitude) as latitude,
        AVG(longitude) as longitude,
        CURRENT_TIMESTAMP() AS EDW_PUBLCN_ID
    FROM 
        staging.listing
    WHERE
        neighbourhood_cleansed NOT IN (SELECT neighbourhood_name FROM dwh.dim_neighbourhood)
    GROUP BY 
        neighbourhood_cleansed
)

-- Insert new neighbourhoods into dwh.dim_neighbourhood
INSERT INTO dwh.dim_neighbourhood
SELECT
    ROW_NUMBER() OVER(ORDER BY neighbourhood_name) + COALESCE((SELECT MAX(neighbourhood_id) FROM dwh.dim_neighbourhood), 0) as neighbourhood_id,
    neighbourhood_name,
    neighborhood_overview,
    latitude,
    longitude,
    EDW_PUBLCN_ID
FROM
    NewNeighbourhoods;


num_affected_rows,num_inserted_rows
0,0


In [0]:
%sql
select neighbourhood_id, count(*) 
from dwh.dim_neighbourhood
group by neighbourhood_id
having count(*) > 1

neighbourhood_id,count(1)


####DIM LISTING
the attributes  of the dwh.Dim_Listing table are:

- listing_id: unique identifier for each listing.
- listing_url:  stores the URL of the listing.
- name:  the name or title of the listing.
- description:  a description of the listing.
- neighborhood_overview:  an overview of the neighborhood where the listing is located.
- picture_url: A stores the URL of the listing's main picture or image.
- property_type: the type of property being listed (e.g., apartment, house, etc.).
- room_type: the type of room being listed (e.g., entire home, private room).
- amenities: ists amenities available with the listing.
- price: the price of the listing
- neighbourhood: the specific neighborhood where the listing is located.
- license: A string representing a regulatory or permitting license associated with the listing.
- EDW_PUBLCN_ID: A timestamp, indicating when the record was published to the data warehouse.

In [0]:
%sql
-- Create the dwh.Dim_Listing table
CREATE OR REPLACE TABLE dwh.Dim_Listing(
    listing_id INT NOT NULL,
    listing_url STRING,
    name STRING,
    description STRING,
    picture_url STRING,
    property_type STRING,
    room_type STRING,
    amenities STRING,
    price DECIMAL(10,2),
    neighbourhood_id INTEGER,
    license STRING,
    EDW_PUBLCN_ID TIMESTAMP
)
USING DELTA;

In [0]:
%sql
MERGE INTO dwh.Dim_Listing AS target
USING (
  SELECT 
    l.listing_id,
    l.listing_url,
    l.name,
    l.description,
    l.picture_url,
    l.property_type,
    l.room_type,
    l.amenities,
    l.price,
    n.neighbourhood_id,
    l.license,
    CURRENT_TIMESTAMP() as EDW_PUBLCN_ID
  FROM 
    staging.listing l
    JOIN dwh.dim_neighbourhood n ON l.neighbourhood_cleansed = n.neighbourhood_name
) AS source
ON target.listing_id = source.listing_id
WHEN MATCHED THEN
  UPDATE SET 
    target.listing_url = source.listing_url,
    target.name = source.name,
    target.description = source.description,
    target.picture_url = source.picture_url,
    target.property_type = source.property_type,
    target.room_type = source.room_type,
    target.amenities = source.amenities,
    target.price = source.price,
    target.neighbourhood_id = source.neighbourhood_id,
    target.license = source.license,
    target.EDW_PUBLCN_ID = source.EDW_PUBLCN_ID
WHEN NOT MATCHED THEN
  INSERT (
    listing_id,
    listing_url,
    name,
    description,
    picture_url,
    property_type,
    room_type,
    amenities,
    price,
    neighbourhood_id,
    license,
    EDW_PUBLCN_ID
  )
  VALUES (
    source.listing_id,
    source.listing_url,
    source.name,
    source.description,
    source.picture_url,
    source.property_type,
    source.room_type,
    source.amenities,
    source.price,
    source.neighbourhood_id,
    source.license,
    source.EDW_PUBLCN_ID
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2069,0,0,2069


In [0]:
%sql
--check for duplicate records
select listing_id,count(*) 
from dwh.Dim_Listing
group by all
having count(*) > 1;

####DIM_LISTING_CALENDAR
the attributes of the dwh.dim_listing_calendar table are:

- listing_id: the unique identifier for each listing. It is set as NOT NULL

- date: the date associated with the calendar entry. is also set as NOT NULL.

- available:  indicating whether the listing is available on the specified date. It can have values of TRUE (available) or FALSE (not available).

- price: the price of the listing for the specified date. 

- adjusted_price: the specified date, which might differ from the regular price due to promotions, discounts
- minimum_nights: the minimum number of nights that a guest can book the listing for.

- maximum_nights: he maximum number of nights that a guest can book the listing for.

- EDW_PUBLCN_ID: indicating when the record was published or added to the data warehouse.

In [0]:
%sql
CREATE OR REPLACE TABLE dwh.dim_listing_calendar(
    listing_id INT NOT NULL,
    date DATE NOT NULL,
    available BOOLEAN,
    price DECIMAL(10, 2),
    adjusted_price DECIMAL(10, 2),
    minimum_nights INT,
    maximum_nights INT,
    EDW_PUBLCN_ID TIMESTAMP
)
USING DELTA;

In [0]:
%sql
MERGE INTO dwh.dim_listing_calendar AS target
USING (
    SELECT 
        listing_id,
        CAST(date AS DATE) AS date,
        CASE WHEN available = 't' THEN TRUE 
             WHEN available = 'f' THEN FALSE  
             ELSE NULL END AS available,
        CAST(REPLACE(price, '$', '') AS DECIMAL(10, 2)) AS price,
        CAST(REPLACE(adjusted_price, '$', '') AS DECIMAL(10, 2)) AS adjusted_price,
        minimum_nights,
        maximum_nights,
        CURRENT_TIMESTAMP() AS EDW_PUBLCN_ID
    FROM raw.calendar
    WHERE listing_id IS NOT NULL
) AS source
ON target.listing_id = source.listing_id AND target.date = source.date
WHEN MATCHED THEN
    UPDATE SET 
        target.available = source.available,
        target.price = source.price,
        target.adjusted_price = source.adjusted_price,
        target.minimum_nights = source.minimum_nights,
        target.maximum_nights = source.maximum_nights,
        target.EDW_PUBLCN_ID = source.EDW_PUBLCN_ID
WHEN NOT MATCHED THEN
    INSERT (listing_id,
             date, 
             available, 
             price, 
             adjusted_price, 
             minimum_nights, 
             maximum_nights, 
             EDW_PUBLCN_ID)
    VALUES (source.listing_id, source.date, source.available, source.price, source.adjusted_price, source.minimum_nights, source.maximum_nights, source.EDW_PUBLCN_ID);



In [0]:
%sql
--check for duplicate records
select listing_id, date,count(*) 
from dwh.dim_calendar
group by all
having count(*) > 1;

####dwh.fct_ratings_reviews
table is our first fact table and captures the review scores and related metrics for Airbnb listings.

- listing_id: Unique ID for listings.
- host_id: ID for the host of the listing.
- neighbourhood_id: ID for the neighbourhood.
- property_type: Type of the property.
- review_scores_rating to review_scores_value: Various review scores.
- price: Listing price.
- reviews_per_month: Average reviews per month.
- number_of_reviews to number_of_reviews_l30d: Count of reviews in different timeframes.
- first_review_date: Date of the first review.
- last_review_date: Date of the most recent review.
- EDW_PUBLCN_ID: Record insertion timestamp.

In [0]:
%sql
CREATE OR REPLACE TABLE dwh.fct_ratings_reviews (
    listing_id INT NOT NULL,
    host_id INT,
    neighbourhood_id INT,
    property_type STRING,
    review_scores_rating FLOAT,
    review_scores_accuracy FLOAT,
    review_scores_cleanliness FLOAT,
    review_scores_checkin FLOAT,
    review_scores_communication FLOAT,
    review_scores_location FLOAT,
    review_scores_value FLOAT,
    price FLOAT,
    reviews_per_month FLOAT,
    number_of_reviews INT,
    number_of_reviews_ltm INT,
    number_of_reviews_l30d INT,
    first_review_date DATE,
    last_review_date DATE,
    EDW_PUBLCN_ID TIMESTAMP
)
USING DELTA;

In [0]:
%sql
-- Merge into fact_ratings_reviews based on processed data from staging.listing and dwh.dim_neighbourhood
MERGE INTO dwh.fct_ratings_reviews AS target
USING (
    SELECT
        l.listing_id,
        l.host_id,
        n.neighbourhood_id,
        l.property_type,
        l.review_scores_rating,
        l.review_scores_accuracy,
        l.review_scores_cleanliness,
        l.review_scores_checkin,
        l.review_scores_communication,
        l.review_scores_location,
        l.review_scores_value,
        l.price,
        l.reviews_per_month,
        l.number_of_reviews,
        l.number_of_reviews_ltm,
        l.number_of_reviews_l30d,
        l.first_review_date,
        l.last_review_date,
        current_timestamp() as EDW_PUBLCN_ID
    FROM 
        staging.listing l
    JOIN 
        dwh.dim_neighbourhood n ON l.neighbourhood_cleansed = n.neighbourhood_name
) AS source
ON target.listing_id = source.listing_id
WHEN MATCHED THEN
    UPDATE SET 
        target.host_id = source.host_id,
        target.neighbourhood_id = source.neighbourhood_id,
        target.property_type = source.property_type,
        target.review_scores_rating = source.review_scores_rating,
        target.review_scores_accuracy = source.review_scores_accuracy,
        target.review_scores_cleanliness = source.review_scores_cleanliness,
        target.review_scores_checkin = source.review_scores_checkin,
        target.review_scores_communication = source.review_scores_communication,
        target.review_scores_location = source.review_scores_location,
        target.review_scores_value = source.review_scores_value,
        target.price = source.price,
        target.reviews_per_month = source.reviews_per_month,
        target.number_of_reviews = source.number_of_reviews,
        target.number_of_reviews_ltm = source.number_of_reviews_ltm,
        target.number_of_reviews_l30d = source.number_of_reviews_l30d,
        target.first_review_date = source.first_review_date,
        target.last_review_date = source.last_review_date,
        target.EDW_PUBLCN_ID = source.EDW_PUBLCN_ID
WHEN NOT MATCHED THEN
    INSERT (
        listing_id,
        host_id,
        neighbourhood_id,
        property_type,
        review_scores_rating,
        review_scores_accuracy,
        review_scores_cleanliness,
        review_scores_checkin,
        review_scores_communication,
        review_scores_location,
        review_scores_value,
        price,
        reviews_per_month,
        number_of_reviews,
        number_of_reviews_ltm,
        number_of_reviews_l30d,
        first_review_date,
        last_review_date,
        EDW_PUBLCN_ID
    )
    VALUES (
        source.listing_id,
        source.host_id,
        source.neighbourhood_id,
        source.property_type,
        source.review_scores_rating,
        source.review_scores_accuracy,
        source.review_scores_cleanliness,
        source.review_scores_checkin,
        source.review_scores_communication,
        source.review_scores_location,
        source.review_scores_value,
        source.price,
        source.reviews_per_month,
        source.number_of_reviews,
        source.number_of_reviews_ltm,
        source.number_of_reviews_l30d,
        source.first_review_date,
        source.last_review_date,
        source.EDW_PUBLCN_ID
    )
;


In [0]:
%sql
--check duplicates
select listing_id, host_id, count(*) 
from dwh.fct_ratings_reviews
group by all
having count(*) > 1

#### fct_reviews: 
table captures individual reviews left by Airbnb users.

- review_id: Unique ID for reviews.
- listing_id: Reference to the listing.
- reviewer_id: ID of the reviewer.
- review_date: Date of the review.
- comments: Reviewer's comments.
- EDW_PUBLCN_ID: Record insertion timestamp.


In [0]:
%sql
CREATE TABLE dwh.fct_reviews (
    review_id STRING  NOT NULL,  --PRIMARY KEY,(not supported) 
    listing_id INTEGER,
    reviewer_id STRING,
    review_date DATE,
    comments STRING,
    EDW_PUBLCN_ID TIMESTAMP
);


To populate the dwh.fact_reviews table, we'll merge data from the staging.review table into it. This approach ensures no duplicates and allows for easy updates in the future:

In [0]:
%sql
MERGE INTO dwh.fct_reviews AS target
USING (
    SELECT
        review_id,
        listing_id,
        reviewer_id,
        review_date,
        comments,
        current_timestamp() AS EDW_PUBLCN_ID
    FROM 
        staging.review
) AS source
ON target.review_id = source.review_id
WHEN MATCHED THEN
    UPDATE SET 
        target.listing_id = source.listing_id,
        target.reviewer_id = source.reviewer_id,
        target.review_date = source.review_date,
        target.comments = source.comments,
        target.EDW_PUBLCN_ID = source.EDW_PUBLCN_ID
WHEN NOT MATCHED THEN
    INSERT *
;


In [0]:
%sql
--checking for duplicates
select listing_id, review_id, count(*)
from dwh.fct_reviews
group by all
having count(*) > 1

In [0]:
%sql
CREATE OR REPLACE TABLE dwh.dim_reviewer (
    reviewer_id INT ,   -- Identifier for the reviewer
    reviewer_name STRING,           -- Name of the reviewer
    EDW_PUBLCN_ID TIMESTAMP
);


In [0]:
%sql
MERGE INTO dwh.dim_reviewer AS target
USING (
    SELECT DISTINCT
        reviewer_id,
        reviewer_name,
        current_timestamp() AS EDW_PUBLCN_ID
    FROM
        staging.review
) AS source
ON target.reviewer_id = source.reviewer_id
WHEN MATCHED THEN
    UPDATE SET 
        target.reviewer_name = source.reviewer_name,
        target.EDW_PUBLCN_ID = source.EDW_PUBLCN_ID
WHEN NOT MATCHED THEN
    INSERT (reviewer_id, reviewer_name, EDW_PUBLCN_ID)
    VALUES (source.reviewer_id, source.reviewer_name, source.EDW_PUBLCN_ID);


###REPORTING
The reporting database is designed for quick data retrieval for analysis. Views play a key role because they:

- Simplify data presentation.
- Merge data from multiple sources.
- Mask sensitive data.
- Ensure consistent data access.

Essentially, views optimize and secure data for better reporting outcomes.

#### reporting.neighborhood_comparison
Attributes for the view:

- neighbourhood_name: The name of the neighborhood.
- avg_rating: Average rating score of listings in the neighborhood.
- avg_price: Average price of listings in the neighborhood.
- most_common_property_type: Most prevalent property type in the neighborhood based on the number of listings.

In [0]:
%sql

CREATE OR REPLACE VIEW reporting.neighborhood_comparison AS

WITH PropertyTypeCounts AS (
    SELECT
        neighbourhood_name,
        property_type,
        COUNT(*) AS property_type_count
    FROM
        dwh.fct_ratings_reviews as rf
    JOIN
        dwh.dim_neighbourhood  as n 
        ON rf.neighbourhood_id = n.neighbourhood_id
    GROUP BY
        neighbourhood_name,
        property_type
)

, RankedPropertyTypes AS (
    SELECT
        neighbourhood_name,
        property_type,
        property_type_count,
        RANK() OVER (PARTITION BY neighbourhood_name ORDER BY property_type_count DESC) AS ranking
    FROM
        PropertyTypeCounts
)

SELECT
    n.neighbourhood_name,
    round(AVG(f.review_scores_rating), 2) as avg_rating,
    round(AVG(f.price), 2) as avg_price,
    p.property_type as most_common_property_type,
    latitude, 
    longitude   
FROM
    dwh.fct_ratings_reviews f
JOIN
    dwh.dim_neighbourhood n ON f.neighbourhood_id = n.neighbourhood_id
JOIN
    RankedPropertyTypes p ON n.neighbourhood_name = p.neighbourhood_name AND p.ranking = 1
WHERE
    f.review_scores_rating IS NOT NULL AND f.price IS NOT NULL
GROUP BY
    n.neighbourhood_name,
    p.property_type,
    n.latitude,
    n.longitude;


In [0]:
%sql
select * from reporting.neighborhood_comparison

neighbourhood_name,avg_rating,avg_price,most_common_property_type,latitude,longitude
Satigny,4.82,141.0,Private room in apartment,46.21358108520508,6.035404014587402
Chancy,4.84,50.0,Private room in bed and breakfast,46.151371002197266,5.973229885101318
Hermance,4.75,209.25,Entire apartment,46.29617919921875,6.244332027435303
Confignon,4.69,93.5,Entire apartment,46.17920608520508,6.085241985321045
Bernex,1.75,74.67,Private room in apartment,46.17865753173828,6.07257878780365
Soral,4.83,70.0,Entire apartment,46.144265747070314,6.046381950378418
Vandoeuvres,5.0,335.0,Entire house,46.21209907531738,6.2081098556518555
Veyrier,4.79,80.7,Private room in apartment,46.17028359805836,6.17012882232666
Anières,4.99,137.5,Entire house,46.27988052368164,6.2260425090789795
Soral,4.83,70.0,Private room in bed and breakfast,46.144265747070314,6.046381950378418


#### reporting.rating_factors
Attributes for the  view:

- property_type: Type of the property.
- avg_rating: Average rating score for the property type.
- avg_price: Average price for the property type.
- number_of_listings: Count of listings for each property type.

In [0]:
%sql
--Factors for higher rating
CREATE OR REPLACE VIEW reporting.rating_factors AS
SELECT 
    property_type,
    AVG(review_scores_rating)::DECIMAL(4,2) as avg_rating,
    AVG(price)::DECIMAL(10,2) as avg_price,
    COUNT(listing_id) as number_of_listings
FROM 
    dwh.fct_ratings_reviews
WHERE review_scores_rating  IS NOT NULL
  AND price IS NOT NULL
GROUP BY 
    property_type;

In [0]:
%sql
select * from reporting.rating_factors

property_type,avg_rating,avg_price,number_of_listings
Private room in loft,4.90,118.25,4
Farm stay,4.00,219.00,1
Entire apartment,4.63,130.54,848
Entire house,4.56,246.66,29
Shared room in condominium,5.00,133.00,1
Room in boutique hotel,4.30,106.50,14
Private room in casa particular,4.42,67.00,9
Entire guesthouse,4.80,87.56,9
Private room in nature lodge,4.63,70.00,1
Entire guest suite,4.27,96.25,4


#### reporting.price_rating_correlation
Attributes for the  view:

- listing_id: Unique identifier for the listing.
- average_price: Average price for the listing.
- average_rating: Average rating score for the listing.

In [0]:
%sql
CREATE OR REPLACE VIEW reporting.price_rating_correlation AS
SELECT 
    listing_id,
    AVG(price)::DECIMAL(10,2) as average_price,  
    AVG(review_scores_rating)::DECIMAL(4,2) as average_rating  
FROM 
    dwh.fact_ratings_reviews
WHERE review_scores_rating  IS NOT NULL
  AND price IS NOT NULL
GROUP BY 
    listing_id;

In [0]:
%sql
select * from reporting.price_rating_correlation

listing_id,average_price,average_rating
7297517,64.00,4.37
15286007,90.00,4.92
29254763,154.00,4.94
34107998,110.00,5.00
36587460,65.00,5.00
40082506,100.00,4.59
40837539,110.00,4.00
47778061,219.00,4.00
48342247,92.00,3.00
6125767,140.00,4.50


In [0]:
%sql
SELECT 
    CORR(average_price, average_rating) as correlation_coefficient
FROM 
    reporting.price_rating_correlation;


Given the value 0.030060627628897704:

It's very close to 0, indicating a very weak positive correlation between average_price and average_rating.
This means that there's a very slight tendency that as the price increases, the rating also increases. However, the correlation is so weak that it's almost negligible.